# 1. Creación del Dataset Planes de Estudio

Fuente [Programación de matricula SUM](https://sum.unmsm.edu.pe/loginWebSum/planes.htm)
- Este archivo genera:
    1. plan_estudios.csv
    2. ciclo_asignatura_base.csv (Requiere trabajo manual)

## Importamos liberías

In [1]:
import pdfplumber
import pandas as pd
import re

## Entendemos como se esta leyendo el pdf y sus tablas

In [2]:
pdf = pdfplumber.open("..\\Data\\Origen\\Plan-Estudios.pdf")
page = pdf.pages[0]
page.extract_tables()

[[['Esp.', 'Asignatura', 'Créd.', 'Tipo', 'Grupo', 'Pre-Requisito', 'Grupo'],
  ['0', 'INE001 - PROCESO CULTURAL ANDINO', '2.0', 'E', 'GEG', '', '--'],
  ['0', 'INE002 - PROGRAMACIÓN Y COMPUTACIÓN', '2.0', 'E', 'GEG', '', '--'],
  ['0', 'INE003 - DIBUJO TÉCNICO', '2.0', 'E', 'GEG', '', '--'],
  ['0',
   'INE004 - INGLÉS PARA ESCRITURA\nACADÉMICA',
   '2.0',
   'E',
   'GEG',
   '',
   '--'],
  ['0', 'INE005 - MATLAB', '2.0', 'E', 'GEG', '', '--'],
  ['0', 'INE006 - CÁLCULOS BÁSICOS EN QUÍMICA', '2.0', 'E', 'GEG', '', '--'],
  ['0',
   'INE007 - SEGURIDAD E HIGIENE EN\nLABORATORIO',
   '2.0',
   'E',
   'GEG',
   '',
   '--'],
  ['0',
   'INE008 - FUNDAMENTOS DE RIESGOS DE\nDESASTRES Y CAMBIO CLIMÁTICO',
   '2.0',
   'E',
   'GEG',
   '',
   '--'],
  ['0', 'INE009 - GEOGRAFIA ECONÓMICA DEL PERÚ', '2.0', 'E', 'GEG', '', '--'],
  ['0',
   'INE010 - CIUDADANÍA Y DERECHOS\nFUNDAMENTALES',
   '2.0',
   'E',
   'GEG',
   '',
   '--'],
  ['0',
   'INE011 - TALLER DE ELECTRICIDAD Y\nELECTRONICA

## Código extractor || Planes de estudio

In [3]:
pdf_path="..\\Data\\Origen\\Plan-Estudios.pdf"

In [4]:
data = []
ciclo_actual = None

# Encabezado estándar que usaremos en páginas sin encabezado visible
encabezado_oficial = ['Esp.', 'Asignatura', 'Créd.', 'Tipo', 'Grupo', 'Pre-Requisito', 'Grupo']

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        page_num = page.page_number
        text = page.extract_text()

        # Buscar el ciclo en el texto
        for line in text.split('\n'):
            if line.strip().startswith("CICLO"):
                try:
                    ciclo_actual = int(line.strip().split()[-1])
                except:
                    ciclo_actual = None

        # Extraer todas las tablas
        tables = page.extract_tables()

        for table in tables:
            if not table or len(table) < 2:
                continue

            # Insertar encabezado si estamos en páginas sin encabezado visual (todas menos 1 y 4)
            if page_num not in [1, 4]:
                table.insert(0, encabezado_oficial)

            for row in table[1:]:  # Saltar encabezado
                if not any(row):
                    continue
                if row[0] and "esp" in row[0].lower():
                    continue

                esp = row[0]
                asignatura = row[1].replace('\n', ' ').strip() if row[1] else None
                creditos = row[2]
                tipo = row[3]
                grupo = row[4] if len(row) > 4 else None
                prerequisito = row[5].replace('\n', ' ').strip() if len(row) > 5 and row[5] else None
                grupo_prereq = row[6] if len(row) > 6 else None

                data.append({
                    "Ciclo": ciclo_actual,
                    "Especialidad": esp,
                    "Asignatura": asignatura,
                    "Créditos": creditos,
                    "Tipo": tipo,
                    "Grupo": grupo,
                    "Pre-Requisito": prerequisito,
                    "Grupo Pre-Requisito": grupo_prereq
                })




In [5]:
# Crear DataFrame
df = pd.DataFrame(data)

# Convertir columnas a tipo numérico donde corresponda
df['Créditos'] = pd.to_numeric(df['Créditos'], errors='coerce')
df['Ciclo'] = pd.to_numeric(df['Ciclo'], errors='coerce')

df.head()


,Ciclo,Especialidad,Asignatura,Créditos,Tipo,Grupo,Pre-Requisito,Grupo Pre-Requisito
0,2,0,INE001 - PROCESO CULTURAL ANDINO,2.0,E,GEG,None,--
1,2,0,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2.0,E,GEG,None,--
2,2,0,INE003 - DIBUJO TÉCNICO,2.0,E,GEG,None,--
3,2,0,INE004 - INGLÉS PARA ESCRITURA ACADÉMICA,2.0,E,GEG,None,--
4,2,0,INE005 - MATLAB,2.0,E,GEG,None,--


Podemos usar esto para asignar código de ciclo dentro de un csv

In [6]:
#df.to_csv("..\\Data\\Extraidos\\plan_estudios.csv",sep=";",encoding='utf-8-sig')

Luego de quitar duplicados por asignatura, podemos asignar el ciclo al que pertenecen los cursos. De ahí creamos el dataset ciclo_asignatura.csv

# 2. Creación del DataSet CicloAsignatura

In [7]:
df_ciclo=pd.read_csv("..\\Data\\Extraidos\\ciclo_asignatura.csv",sep=";",encoding='utf-8-sig')

In [8]:
df_ciclo.head()

,Ciclo,Asignatura
0,1,INE001 - PROCESO CULTURAL ANDINO
1,1,INE002 - PROGRAMACIÓN Y COMPUTACIÓN
2,1,INE003 - DIBUJO TÉCNICO
3,1,INE004 - INGLÉS PARA ESCRITURA ACADÉMICA
4,1,INE005 - MATLAB


Contiene la información del ciclo al que pertenece cada curso

In [9]:
#df_ciclo[["CodAsignatura","NomAsignatura"]]=df_ciclo['Asignatura'].str.split(' - ', n=1, expand=True)
#df_ciclo=df_ciclo.drop(columns=["Asignatura"])
#df_ciclo

In [10]:
#df_ciclo.to_csv("..\\Data\\Extraidos\\ciclo_asignatura.csv",sep=";",encoding='utf-8-sig',index=False)

In [11]:
df=df.drop(columns=["Especialidad"])

In [12]:
df=df.merge(df_ciclo,how="left",on="Asignatura")

In [13]:
df["Ciclo"]=df["Ciclo_y"]
df=df.drop(columns=["Ciclo_y","Ciclo_x"])
df.head()

,Asignatura,Créditos,Tipo,Grupo,Pre-Requisito,Grupo Pre-Requisito,Ciclo
0,INE001 - PROCESO CULTURAL ANDINO,2.0,E,GEG,None,--,1
1,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2.0,E,GEG,None,--,1
2,INE003 - DIBUJO TÉCNICO,2.0,E,GEG,None,--,1
3,INE004 - INGLÉS PARA ESCRITURA ACADÉMICA,2.0,E,GEG,None,--,1
4,INE005 - MATLAB,2.0,E,GEG,None,--,1


# 3. Normalización del campo Asignatura y Pre-Requisito

## Separamos Asignatura en codigo asignatura y nombre de asignatura

In [14]:
df[["CodAsignatura","NomAsignatura"]]=df['Asignatura'].str.split(' - ', n=1, expand=True)
df=df.drop(columns=["Asignatura"])
df.head()

,Créditos,Tipo,Grupo,Pre-Requisito,Grupo Pre-Requisito,Ciclo,CodAsignatura,NomAsignatura
0,2.0,E,GEG,None,--,1,INE001,PROCESO CULTURAL ANDINO
1,2.0,E,GEG,None,--,1,INE002,PROGRAMACIÓN Y COMPUTACIÓN
2,2.0,E,GEG,None,--,1,INE003,DIBUJO TÉCNICO
3,2.0,E,GEG,None,--,1,INE004,INGLÉS PARA ESCRITURA ACADÉMICA
4,2.0,E,GEG,None,--,1,INE005,MATLAB


## Separamos Pre-Requisito en Codigo Pre-Requisito y Nombre Pre-Requisito

In [15]:
df[["Cod Pre-Requisito","Nom Pre-Requisito"]]=df['Pre-Requisito'].str.split(' - ', n=1, expand=True)
df=df.drop(columns=["Pre-Requisito"])

In [16]:
df=df[["Ciclo","Créditos","CodAsignatura","NomAsignatura","Cod Pre-Requisito","Nom Pre-Requisito"]]
df.head()

,Ciclo,Créditos,CodAsignatura,NomAsignatura,Cod Pre-Requisito,Nom Pre-Requisito
0,1,2.0,INE001,PROCESO CULTURAL ANDINO,None,None
1,1,2.0,INE002,PROGRAMACIÓN Y COMPUTACIÓN,None,None
2,1,2.0,INE003,DIBUJO TÉCNICO,None,None
3,1,2.0,INE004,INGLÉS PARA ESCRITURA ACADÉMICA,None,None
4,1,2.0,INE005,MATLAB,None,None


In [ ]:
df.to_csv("..\\Data\\Extraidos\\plan_estudios_completo.csv",sep=";",encoding='utf-8-sig',index=False)